In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
# 규원님이 전처리 완료하신 최종 데이터 (+라벨까지 붙은)
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/data/dataset_210626_215600.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
len(df)

74779

In [ ]:
df.head()

,gender,name,accords,review,tokenized,only_english,longer_than_2_A,stopwords_removed,lemmatizated,label
0,female,Alien Mugler for women,"['white floral', 'amber', 'woody']","Got a sample of this today, and my 9 year old ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'a', 'sample', 'of', 'this', 'today', ...","['got', 'sample', 'this', 'today', 'and', 'yea...","['got', 'sample', 'today', 'year', 'old', 'dau...","['get', 'sample', 'today', 'year', 'old', 'dau...",2.0
1,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",First impressions. Test on blotter.\r\nI have ...,"['first', 'impressions.', 'test', 'on', 'blott...","['first', 'impressions', 'test', 'on', 'blotte...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impressions', 'test', 'blotter', 'h...","['first', 'impression', 'test', 'blotter', 'he...",2.0
2,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",This perfume reminds me of my best friend. Act...,"['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'me', 'of', 'my...","['this', 'perfume', 'reminds', 'best', 'friend...","['perfume', 'reminds', 'best', 'friend', 'actu...","['perfume', 'reminds', 'best', 'friend', 'actu...",2.0
3,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Imagine tripping over your own feet and fallin...,"['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'over', 'your', 'own',...","['imagine', 'tripping', 'feet', 'falling', 'fa...","['imagine', 'trip', 'foot', 'fall', 'face', 'f...",2.0
4,female,Alien Mugler for women,"['white floral', 'amber', 'woody']",Gorgeous Gorgeous Blend ..\r\nLove the scent.....,"['gorgeous', 'gorgeous', 'blend', '..', 'love'...","['gorgeous', 'gorgeous', 'blend', '', 'love', ...","['gorgeous', 'gorgeous', 'blend', 'love', 'the...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...","['gorgeous', 'gorgeous', 'blend', 'love', 'sce...",2.0


본 데이터는 모두 string 형태라 전처리 필요

In [ ]:
sent_text = df['lemmatizated']

In [ ]:
import nltk
# nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     tokens = ' '.join([w for w in tokens.split() if len(w)>=3])
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

In [ ]:
for line in result[:3]:
  print(line)

['get', 'sample', 'today', 'year', 'old', 'daughter', 'think', 'smell', 'spray', 'card', 'rotten', 'fish', 'nearly', 'throw', 'immediately', 'however', 'later', 'even', 'decide', 'give', 'fair', 'trial', 'spray', 'crook', 'elbow', 'rot', 'fish', 'smell', 'time', 'get', 'definite', 'bit', 'funk', 'almost', 'urine', 'scent', 'minute', 'blossom', 'gorgeous', 'smooth', 'woody', 'ambery', 'clean', 'warm', 'jasmine', 'daughter', 'didnt', 'even', 'believe', 'tell', 'fragrance', 'smell', 'earlier', 'lol', 'immediately', 'comment', 'fresh', 'clean', 'vibe', 'agree', 'soapy', 'hint', 'powdery', 'way', 'obsess', 'cant', 'stop', 'smelling', 'arm', 'glad', 'give', 'try', 'wait', 'buy', 'full', 'bottle']
['first', 'impression', 'test', 'blotter', 'heard', 'smell', 'like', 'bag', 'spray', 'like', 'gabriella', 'sabatini', 'perfume', 'everybody', 'past', 'woman', 'wear', 'smell', 'addictive', 'plasticky', 'beautiful', 'amaze', 'projection', 'longevity', 'woman', 'strong', 'personality', 'wear', 'may', 

### Word2Vec 훈련시키기

하이퍼 파라미터
* size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
* window = 컨텍스트 윈도우 크기
* min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
* workers = 학습을 위한 프로세스 수
* sg = 0은 CBOW, 1은 Skip-gram.

In [ ]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.callbacks import CallbackAny2Vec
class callback(CallbackAny2Vec): 
  """Callback to print loss after each epoch."""
  def __init__(self):
    self.epoch = 0
    self.loss_to_be_subed = 0
  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    loss_now = loss - self.loss_to_be_subed
    self.loss_to_be_subed = loss
    print('Loss after epoch {}: {}'.format(self.epoch, loss_now))
    self.epoch += 1


model = Word2Vec(sentences=result, size=100, window=5, min_count=2, workers=4, sg=1, compute_loss=True, iter=5, callbacks=[callback()])

Loss after epoch 0: 9404458.0
Loss after epoch 1: 7431250.0
Loss after epoch 2: 6613720.0
Loss after epoch 3: 6653580.0
Loss after epoch 4: 5350292.0


### 1 Eopch만 수행 후 결과

In [ ]:
model_result = model.wv.most_similar("fruity")
print(model_result)

[('rosey', 0.7871946692466736), ('candylike', 0.7860710620880127), ('citrusy', 0.777799665927887), ('pineappley', 0.7775022387504578), ('citrus', 0.7759712934494019), ('sweet', 0.7713905572891235), ('birchy', 0.7667485475540161), ('citrussy', 0.7658706903457642), ('peachy', 0.7642098665237427), ('orangey', 0.7631978392601013)]


In [ ]:
model_result = model.wv.most_similar("aquatic")
print(model_result)

[('marine', 0.8327802419662476), ('oceanic', 0.8117033243179321), ('ozonic', 0.7881015539169312), ('watery', 0.7626587152481079), ('citrusy', 0.7545432448387146), ('acquatic', 0.7532049417495728), ('fougre', 0.7523386478424072), ('fougere', 0.7428005933761597), ('herbaceous', 0.7371671199798584), ('citrus', 0.7368615865707397)]


In [ ]:
model_result = model.wv.most_similar("tobacco")
print(model_result)

[('vanille', 0.7508845925331116), ('pipe', 0.7115957140922546), ('smoke', 0.6999630331993103), ('tabacco', 0.690207302570343), ('rum', 0.6827375888824463), ('tobbaco', 0.6776096224784851), ('booze', 0.6739352941513062), ('honey', 0.6658722162246704), ('cinnamon', 0.6556151509284973), ('cigar', 0.6550647020339966)]


In [ ]:
model_result = model.wv.most_similar("floral")
print(model_result)

[('flowery', 0.8187991380691528), ('florals', 0.7975499033927917), ('herbaceous', 0.7875528931617737), ('rosey', 0.7831542491912842), ('woodyness', 0.7713372707366943), ('peachy', 0.7677422761917114), ('powdery', 0.7671135663986206), ('gingery', 0.7654973864555359), ('incenselike', 0.7648699879646301), ('woodymusky', 0.764728307723999)]


In [ ]:
model_result = model.wv.most_similar("orange")
print(model_result)

[('blossom', 0.8139773607254028), ('mandarine', 0.8006924390792847), ('mandarin', 0.7795277833938599), ('oranges', 0.7774101495742798), ('grapefruit', 0.7760952115058899), ('tangerine', 0.7698485851287842), ('blackcurrent', 0.7682220935821533), ('mimosa', 0.765866756439209), ('lime', 0.7596749663352966), ('cassia', 0.7576935291290283)]


### 5 Epoch 수행 후 결과

In [ ]:
model_result = model.wv.most_similar("fruity")
print(model_result)

[('citrussy', 0.7941665649414062), ('candylike', 0.7924445867538452), ('spicysweet', 0.7918358445167542), ('pineappley', 0.7855466604232788), ('citrusy', 0.783761739730835), ('lychee', 0.7803213000297546), ('fruty', 0.7784193754196167), ('citrus', 0.777553141117096), ('orangey', 0.7774918079376221), ('birchy', 0.776764988899231)]


In [ ]:
model_result = model.wv.most_similar("aquatic")
print(model_result)

[('marine', 0.8482732772827148), ('oceanic', 0.8091974258422852), ('citrus', 0.7736861109733582), ('citrusy', 0.7645018100738525), ('watery', 0.7624021768569946), ('ozonic', 0.757739782333374), ('fresh', 0.7573273181915283), ('acquatic', 0.755082905292511), ('beachy', 0.7424953579902649), ('citric', 0.7383270263671875)]


In [ ]:
model_result = model.wv.most_similar("tobacco")
print(model_result)

[('vanille', 0.7691082954406738), ('pipe', 0.7288037538528442), ('honey', 0.695122241973877), ('tabacco', 0.6913232207298279), ('cinnamon', 0.6847171187400818), ('booze', 0.6835865378379822), ('spice', 0.6760627031326294), ('rum', 0.6702158451080322), ('tobbaco', 0.6639431118965149), ('smoke', 0.6603217720985413)]


In [ ]:
model_result = model.wv.most_similar("floral")
print(model_result)

[('flowery', 0.8534170389175415), ('florals', 0.8268424272537231), ('overlay', 0.7807486057281494), ('gingery', 0.7782377600669861), ('undertones', 0.7772520184516907), ('rosey', 0.7762121558189392), ('acquatic', 0.7722552418708801), ('underpin', 0.7709872126579285), ('powdery', 0.7703284025192261), ('herbaceous', 0.7673629522323608)]


In [ ]:
model_result = model.wv.most_similar("orange")
print(model_result)

[('blossom', 0.8431805968284607), ('mandarin', 0.8259761333465576), ('grapefruit', 0.7909433245658875), ('oranges', 0.7805365324020386), ('mandarine', 0.7777758836746216), ('mimosa', 0.7683936953544617), ('peel', 0.767780065536499), ('lemon', 0.7638107538223267), ('citruses', 0.7636439800262451), ('peppercorn', 0.7616171836853027)]


큰 차이는 없어보이지만 Loss가 줄었으니 이 모델로 저장하겠음

### 모델 저장 및 로드

In [ ]:
model.wv.save_word2vec_format('/content/drive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding/model/w2v_5window') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding/model/w2v_5window') # 모델 로드

In [ ]:
model_result = loaded_model.most_similar("orange")
print(model_result)

[('blossom', 0.8431805968284607), ('mandarin', 0.8259761333465576), ('grapefruit', 0.7909433245658875), ('oranges', 0.7805365324020386), ('mandarine', 0.7777758836746216), ('mimosa', 0.7683936953544617), ('peel', 0.767780065536499), ('lemon', 0.7638107538223267), ('citruses', 0.7636439800262451), ('peppercorn', 0.7616171836853027)]


In [ ]:
print(loaded_model.wv.similarity("ocean", "summer"))

0.55251265


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [ ]:
print(loaded_model.wv.similarity("forest", "woody"))

0.42345577


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [ ]:
print(loaded_model.wv.similarity("jasmine", "floral"))

0.693579


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
